In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from typing import List, Dict
from datetime import datetime, timedelta
sns.set_theme(style="darkgrid")
from calculations.common import (
    prepare_data,
    calculate_session_stats
)
from calculations.web_traffic import (
    aggregate_web_traffic_daily,
    calculate_social_media_correlation,
    aggregate_web_traffic_hourly,
    aggregate_web_traffic_weekday
)

In [ ]:
start_date = '2022-02-17'
end_date = '2022-03-16'
df_raw = pd.read_csv('data/event_data_20220217-20220316.csv')
df_web = prepare_data(df_raw)


df_facebook_reach = pd.read_csv('data/facebook_reach.csv', encoding = "utf-8")
df_instagram_reach = pd.read_csv('data/instagram_reach.csv', encoding = "utf-8")

df_facebook_pageviews = pd.read_csv('data/facebook_pageviews.csv', encoding = "utf-8")
df_instagram_pageviews = pd.read_csv('data/instagram_pageviews.csv', encoding = "utf-8")


In [ ]:
date_preproc = aggregate_web_traffic_daily(df_web)
hour_preproc = aggregate_web_traffic_hourly(df_web)
weekday_preproc = aggregate_web_traffic_weekday(date_preproc)
session_stats = calculate_session_stats(df_web)

In [ ]:
social_media_correlation = calculate_social_media_correlation(
    df_sessions_per_day=date_preproc['Sessioner'],
    df_facebook_reach=df_facebook_reach,
    df_facebook_pageviews=df_facebook_pageviews,
    df_instagram_reach=df_instagram_reach,
    df_instagram_pageviews=df_instagram_pageviews,
    date_days=list(date_preproc['Datum'])
)

# Resultat

In [ ]:
print(f'Trafikdata för hemsidan under perioden {start_date} till {end_date}')

# Sidvisningar, Sessioner & Unika Besökare i snitt över datum
**Sidvisningar (#):** Antalet totala sidvisningar på hemsidan över datum.

**Sessioner (#):** Antalet totala sessioner på hemsidan över datum.

*En session innefattar alla sidvisningar för en enskild besökare där det inte har gått längre än 30 minuter mellan två efterföljande sidvisningar. Om en besökare stannar på en sida under 31 minuter, för att sen går vidare till en annan sida på hemsidan, då startas en ny session. Sessioner som spänner mellan två dygn, alltså över midnatt, kapas vid midnatt och det blir istället två sessioner, en innan och en efter midnatt.*

**Unika Besökare (#):** Antalet unika besökare på hemsidan varje dag.

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,8))
sns.lineplot(x='Datum', y='value', hue='variable', data=pd.melt(
    date_preproc.drop(columns=['Veckodag']), ['Datum']), ax=ax)
plt.xticks(rotation=45); plt.ylabel('Antal')
plt.show()

# Sidvisningar & Sessioner i snitt över veckodag

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,8))
sns.barplot(x='Veckodag', y='value', hue='variable', data=pd.melt(weekday_preproc, ['Veckodag']), ax=ax)
plt.xticks(rotation=45); plt.ylabel('Snitt Antal')
plt.show()

# Sidvisningar & Sessioner i snitt över tid på dygnet

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,8))
sns.barplot(x='Timme', y='value', hue='variable', data=pd.melt(hour_preproc, ['Timme']), ax=ax)
plt.xticks(rotation=45); plt.ylabel('Snitt Antal')
plt.show()

# Antal Sidvisningar och Sid-tid under Sessioner

In [ ]:
session_stats

In [ ]:
print(f"""
Andel av sessioner där besökaren endast besökte en enskild sida:\t {session_stats['one_event_session_ratio']}%
Antal sidor som besökare i snitt besöker under en session:\t\t {session_stats['avg_events_per_session']} stycken
Antal sidor som besökare i snitt besöker under sessioner med 
    fler än en sidvisning:\t\t\t\t\t\t {session_stats['avg_events_per_multievent_sessions']} stycken

Snitt-tid som besökare spenderar
    på varje sida (median):\t\t\t {session_stats['median_time_per_page']} sekunder
    på hemsidan under en session (median):\t {session_stats['median_session_time']} sekunder
""")

# Hemsida Sessioner korrelation Facebook
Hur besökares sessioner på Hemsidan korrelerar (sammanfaller) med Räckvidd och Sidvisningar på Facebook

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,8))
sns.lineplot(x='Datum', y='value', hue='variable', data=pd.melt(social_media_correlation['facebook'], ['Datum']), ax=ax)
plt.xticks(rotation=45); plt.ylabel('Intensitet')
plt.show()

# Hemsida Sessioner korrelation Instagram
Hur besökares sessioner på Hemsidan korrelerar (sammanfaller) med Räckvidd och Sidvisningar på Instagram

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,8))
sns.lineplot(x='Datum', y='value', hue='variable', data=pd.melt(
    social_media_correlation['instagram'], ['Datum']), ax=ax)
plt.xticks(rotation=45); plt.ylabel('Intensitet')
plt.show()